In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType,DoubleType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import regexp_replace
import locale
import sys
import os
import re

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

# Leyendo lista de Colegios y Estudiantes

In [4]:
base = '../Colegios'
base2 = '../SB'
colegios = os.listdir('../Colegios')
estudiantes = os.listdir('../SB')

In [5]:
estudiantes.sort()
colegios.sort()
estuDic = {}
coleDic = {}
for estu in estudiantes:
    estuDic[re.findall('\d+', estu)[1]] = estu

for colegio in colegios:
    coleDic[re.findall('\d+', colegio)[1]] = colegio

In [6]:
# Funciòn para unir cualquier nùmero de dataframes con la misma estructura 
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [7]:
print("Archivos de estudiantes: ", estuDic['20081'],estuDic['20082'])
print("Archivos de colegios: ",coleDic['2008'])

Archivos de estudiantes:  SB11_20081.txt SB11_20082.txt
Archivos de colegios:  SB11-CLASIFI-PLANTELES-2008.txt


In [8]:
columnasPuntos = [
 'cole_cod_dane_institucion',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_ingles',
 'punt_profundizacion',
 'punt_interdisciplinar']

In [9]:
# SE leen los dos archivos y se crean dos dataframes
df1 = spark.read.load(base2 + '/' + estuDic['20081'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬').select(columnasPuntos)
df2 = spark.read.load(base2 + '/' + estuDic['20082'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬').select(columnasPuntos)

In [10]:
# SE unen los dos dataframes
columnasPuntosc = [
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_ingles',
 'punt_profundizacion',
 'punt_interdisciplinar']
df = unionAll(df1,df2)
for cp in columnasPuntosc:
    df = df.withColumn(cp, regexp_replace(cp, ',', '.'))
    df = df.withColumn(cp, df[cp].cast("float"))
    
df1 = df2 = None

In [11]:
print("Numero de filas ", df.count())

Numero de filas  506431


In [12]:
print(df.printSchema())

root
 |-- cole_cod_dane_institucion: string (nullable = true)
 |-- punt_lenguaje: float (nullable = true)
 |-- punt_matematicas: float (nullable = true)
 |-- punt_c_sociales: float (nullable = true)
 |-- punt_filosofia: float (nullable = true)
 |-- punt_biologia: float (nullable = true)
 |-- punt_quimica: float (nullable = true)
 |-- punt_fisica: float (nullable = true)
 |-- punt_ingles: float (nullable = true)
 |-- punt_profundizacion: float (nullable = true)
 |-- punt_interdisciplinar: float (nullable = true)

None


In [13]:
# Contando el numero de estudiantes por instituciòn y mostrandola de forma descendente
dsCountEst = df.groupBy('cole_cod_dane_institucion').count().orderBy('count',ascending=
False)

In [14]:
dsCountEst.show(10,False)

+-------------------------+-----+
|cole_cod_dane_institucion|count|
+-------------------------+-----+
|null                     |6390 |
|105001000108             |1156 |
|105001013340             |1015 |
|111001024732             |852  |
|168001003591             |761  |
|311001105391             |748  |
|111001019411             |657  |
|305001004293             |653  |
|305001003963             |629  |
|105001024571             |610  |
+-------------------------+-----+
only showing top 10 rows



In [15]:
#Se crea una columna con el total de los puntajes   
df= df.withColumn("Suma",df['punt_lenguaje']+df['punt_matematicas']+df['punt_c_sociales']+
                     df['punt_filosofia']+df['punt_biologia']+df['punt_quimica']+df['punt_fisica']+
                     df['punt_ingles']+df['punt_profundizacion']+df['punt_interdisciplinar'])

## Resultados finales por instituciòn

In [16]:
# Còdigo para obtener la media de los resultados de las evaluaciones de todos los estudiantes por instituciòn 
# y posiciòn en orden descendente.
from pyspark.sql.functions import *
from pyspark.sql.window import Window

df = df.groupBy('cole_cod_dane_institucion').mean()
df = df.withColumn("rank", dense_rank().over(Window.orderBy(desc("avg(Suma)"))))
df = df.orderBy('rank',ascending= True)

In [17]:
df.select('cole_cod_dane_institucion','avg(Suma)','rank').show(10,False)

+-------------------------+-----------------+----+
|cole_cod_dane_institucion|avg(Suma)        |rank|
+-------------------------+-----------------+----+
|311848000278             |596.1134561391978|1   |
|311848002424             |591.9240733959057|2   |
|311769003342             |583.8497792119565|3   |
|308001004209             |580.5805244954427|4   |
|368276000826             |579.8268853549299|5   |
|311769000165             |574.3698745920689|6   |
|311848002351             |571.9305555555555|7   |
|311848002262             |571.2892496360922|8   |
|311848000286             |570.2402056884765|9   |
|311001087288             |569.1046746436585|10  |
+-------------------------+-----------------+----+
only showing top 10 rows



## Colegios

In [18]:
# se crea un dataframe con una columna que lo señala el año 2006.
from pyspark.sql.functions import lit
df = df.withColumn('año',lit(2008))
df.show(5,False)

+-------------------------+------------------+---------------------+--------------------+-------------------+------------------+------------------+------------------+-----------------+------------------------+--------------------------+-----------------+----+----+
|cole_cod_dane_institucion|avg(punt_lenguaje)|avg(punt_matematicas)|avg(punt_c_sociales)|avg(punt_filosofia)|avg(punt_biologia)|avg(punt_quimica) |avg(punt_fisica)  |avg(punt_ingles) |avg(punt_profundizacion)|avg(punt_interdisciplinar)|avg(Suma)        |rank|año |
+-------------------------+------------------+---------------------+--------------------+-------------------+------------------+------------------+------------------+-----------------+------------------------+--------------------------+-----------------+----+----+
|311848000278             |63.33923031733586 |75.28846285893367    |58.86615430391752   |67.3742320720966   |64.95115382854755 |62.65769166212816 |59.681923352755035|79.44307826115535|6.247692291553204    

In [19]:
# SE crea Dataframe de colegios
dfc = spark.read.load(base + '/' + coleDic['2008'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='\t')
dfc = dfc.withColumn('COLE_CODIGO_COLEGIO', dfc['COLE_CODIGO_COLEGIO'].cast("string"))
dfc = dfc.withColumn('COLE_CODMPIO_COLEGIO', dfc['COLE_CODMPIO_COLEGIO'].cast("string"))

In [20]:
print(dfc.printSchema())

root
 |-- COLE_CODIGO_COLEGIO: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: string (nullable = true)
 |-- COLE_MPIO_COLEGIO: string (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_INST_JORNADA: string (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_CIENCIAS_SOCIALES: integer (nullable = true)
 |-- COLE_QUIMICA: integer (nullable = true)
 |-- COLE_FISICA: integer (nullable = true)
 |-- COLE_BIOLOGIA: integer (nullable = true)
 |-- COLE_FILOSOFIA: integer (nullable = true)
 |-- COLE_MATEMATICAS: integer (nullable = true)
 |-- COLE_LENGUAJE: integer (nullable = true)
 |-- COLE_INGLES: integer (nullable = true)
 |-- COLE_GEOGRAFIA: string (nullable = true)
 |-- COLE_HISTORIA: string (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)
 |-- COLE_ESTUDIANTES_PRESENT

In [21]:
colsColegio = ['COLE_CODIGO_COLEGIO',
 'COLE_CODMPIO_COLEGIO',
 'COLE_CIENCIAS_SOCIALES',
 'COLE_QUIMICA',
 'COLE_FISICA',
 'COLE_BIOLOGIA',
 'COLE_FILOSOFIA',
 'COLE_MATEMATICAS',
 'COLE_LENGUAJE',
 'COLE_INGLES',
 'COLE_GEOGRAFIA',
 'COLE_HISTORIA']

In [22]:
dfc = dfc.select(colsColegio)

In [23]:
dfc.show(5,False)

+-------------------+--------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|COLE_CODIGO_COLEGIO|COLE_CODMPIO_COLEGIO|COLE_CIENCIAS_SOCIALES|COLE_QUIMICA|COLE_FISICA|COLE_BIOLOGIA|COLE_FILOSOFIA|COLE_MATEMATICAS|COLE_LENGUAJE|COLE_INGLES|COLE_GEOGRAFIA|COLE_HISTORIA|
+-------------------+--------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|265                |5001                |8                     |6           |5          |7            |8             |7               |7            |7          |null          |null         |
|273                |5001                |9                     |9           |8          |8            |8             |8               |8            |9          |null          |null         |
|281                |5001               

In [24]:
print("El numero de colegios es: ", dfc.select('COLE_CODIGO_COLEGIO').count())

El numero de colegios es:  10160


## Anàlisis comparativo entre estudiantes y colegios

In [25]:
dfjoin = df.join(dfc, df.cole_cod_dane_institucion == dfc.COLE_CODIGO_COLEGIO)

In [26]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios:", dfjoin.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios: 0


In [27]:
dfjoin = df.join(dfc, df.cole_cod_dane_institucion == dfc.COLE_CODMPIO_COLEGIO)
dfjoin.count()
print("Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios:", dfjoin.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios: 0


La informaciòn de colegios no coincide con la informaciòn de estudiantes, no existe cole_cod_dane_institucion en el archivo de colegios.